In [1]:
import csv

import numpy as np
%run ../chap01/abalone.ipynb

In [2]:
#메인 함수 정의
def steel_exec(epoch_count=10, mb_size=10, report=1):
    load_steel_dataset()
    init_model()
    train_and_test(epoch_count, mb_size, report)

In [3]:
#데이터 적재 함수 정의
def load_steel_dataset():
    with open('../../data/chap03/faults.csv') as csvfile:
        csvreader = csv.reader(csvfile)
        next(csvreader, None)
        rows = []
        for row in csvreader:
            rows.append(row)
    global data, input_cnt, output_cnt
    input_cnt, output_cnt = 27, 7 #output_cnt -> 철강 표면에 대한 분류 항목 수 7개로 지정
    data = np.asarray(rows, dtype='float32')

In [4]:
#후처리 과정에 대한 순전파와 역전파 함수의 재정의
def forward_postproc(output, y):
    entropy = softmax_cross_entropy_with_logits(y, output)
    loss =  np.mean(entropy)
    return loss, [y, output, entropy]

def backprop_postproc(G_loss, aux):
    y, output, entropy = aux

    g_loss_entropy = 1.0 / np.prod(entropy.shape)
    g_entropy_outut = softmax_cross_entropy_with_logits_derv(y, output)

    G_entropy = g_loss_entropy * G_loss
    G_output = g_entropy_outut * G_entropy

    return G_output

In [5]:
#정확도 계산 함수의 재정의
def eval_accuracy(output, y):
    #argmax() 함수: 후보 항목에 관한 각 로짓값을 담고 있는 벡터에서 가장 큰 값이 어디있는지 확인
    estimate = np.argmax(output, axis=1)
    answer = np.argmax(y, axis=1)
    correct = np.equal(estimate, answer)

    return np.mean(correct)

In [6]:
#소프트맥스 관련 함수 정의
def softmax(x):
    max_elem = np.max(x, axis=1)
    diff = (x.transpose() - max_elem).transpose()
    exp = np.exp(diff)
    sum_exp = np.sum(exp, axis=1)
    probs = (exp.transpose() / sum_exp).transpose()
    return probs

def softmax_derv(x, y):
    mb_size, nom_size = x.shape
    derv = np.ndarray([mb_size, nom_size, nom_size])
    for n in range(mb_size):
        for i in range(nom_size):
            for j in range(nom_size):
                derv[n, i, j] = -y[n, i] * y[n, j]
            derv[n, i, i] += y[n, i]
    return derv

def softmax_cross_entropy_with_logits(labels, logits):
    probs = softmax(logits)
    return -np.sum(labels * np.log(probs+1.0e-10), axis=1)

def softmax_cross_entropy_with_logits_derv(labels, logits):
    return softmax(logits) - labels



In [10]:
steel_exec()

Epoch 1" loss=16.8757, accuracy=0.267/0.297
Epoch 2" loss=15.3456, accuracy=0.334/0.238
Epoch 3" loss=15.2862, accuracy=0.336/0.238
Epoch 4" loss=15.5684, accuracy=0.324/0.440
Epoch 5" loss=15.6873, accuracy=0.319/0.248
Epoch 6" loss=15.6873, accuracy=0.319/0.235
Epoch 7" loss=15.5090, accuracy=0.326/0.361
Epoch 8" loss=15.6576, accuracy=0.320/0.317
Epoch 9" loss=15.1079, accuracy=0.344/0.437
Epoch 10" loss=15.2713, accuracy=0.337/0.496

Final Test: final accuracy = 0.496
